Please make sure that the data files are located in the folder "season_1/training_data/order_data/". Then run all the cells.

In [1]:
import pandas as pd
from os import walk

"""import the order datas into a dataframe""" 

# Use path to data files.
order_path = "season_2/training_data/order_data/" 
_, _, filenames = next(walk(order_path), (None, None, []))

mSeq = len(filenames)
frame = pd.DataFrame()
list_ = []
print filenames


['order_data_2016-01-01', 'order_data_2016-01-02', 'order_data_2016-01-03', 'order_data_2016-01-04', 'order_data_2016-01-05', 'order_data_2016-01-06', 'order_data_2016-01-07', 'order_data_2016-01-08', 'order_data_2016-01-09', 'order_data_2016-01-10', 'order_data_2016-01-11', 'order_data_2016-01-12', 'order_data_2016-01-13', 'order_data_2016-01-14', 'order_data_2016-01-15', 'order_data_2016-01-16', 'order_data_2016-01-17', 'order_data_2016-01-18', 'order_data_2016-01-19', 'order_data_2016-01-20', 'order_data_2016-01-21']


In [3]:
# Uncomment these lines to import all the data 
for i in range(mSeq-1):
    # White Space as Delim, import all the data file.
    df = pd.read_csv(str(order_path) + filenames[i+1],  delim_whitespace = 1, dtype = str, header = None) 
    list_.append(df)

orders= pd.concat(list_)

# Only import the data of the first day
# orders = pd.read_csv(str(order_path) + filenames[1], delim_whitespace = 1, dtype = str,  header = None) 


In [4]:

# Only import the data of the first day
# orders = pd.read_csv(str(order_path) + filenames[1], delim_whitespace = 1, dtype = str,  header = None) 

# Rename all the columns.
orders.columns = ['order_id', 'driver_id', 'passenger_id', 
                  'start_district_hash', 'dest_district_hash', 
                  'price', 'date', 'time']
# Convert the Date and Time column in orders from an object to a datetime value. 
# This makes it easier to work with as a date.
# Extract the year and month from from date_time, and assign them to their own columns.
orders['datetime'] = pd.to_datetime(orders['date'] + ' ' + orders['time'])
orders = orders.drop('date', 1)
orders = orders.drop('time', 1)
orders['day'] = orders["datetime"].dt.day
# Every 10 minutes is considered as a time slot. It can be calculated by following fomula
orders['time_slot'] = orders["datetime"].dt.hour*6 + orders["datetime"].dt.minute//10


In [5]:
"""Categorize the start and destination with its id in """

# Import the hash id into a dictionary
cluster_path = "season_2/training_data/cluster_map/"
cluster = {}
with open(cluster_path + "cluster_map") as f:
    for line in f:
       (key, val) = line.split()
       cluster[str(key)] = val
        
# Update the start and destination district ID
orders['start_district_id'] = orders['start_district_hash'].map(cluster)
orders['dest_district_id'] = orders['dest_district_hash'].map(cluster)

In [6]:
# Take a look at the data. The number of the data collected in each district
# is not evenly distributed. District 51 can be considered as a high occupancied
# zone. We're more likely to dig into this kind of areas.
# Also, interstingly there are more orders in from 12am to 1am, which is not expected.
# Probably because of in this period there are still demands but not enough taxis or 
# other public transportation.

print(orders["start_district_id"].value_counts())
print(orders["time_slot"].value_counts())

51    1215703
23     724035
8      706466
48     518647
28     473581
1      418848
7      394201
37     373459
46     319435
20     263257
12     261927
24     216688
14     214434
21     188375
42     177359
27     157682
4      133162
19     123860
22      96254
26      95761
25      81960
9       76012
29      59258
11      55322
36      54060
2       49432
6       42375
54      39032
35      33547
57      32064
       ...   
39      22637
47      21857
41      20135
16      20084
34      19159
18      18752
33      17309
40      15565
17      13801
56      11125
45      10454
58       7993
5        7678
10       7412
3        7314
59       6441
49       6376
32       6150
65       5849
30       5797
61       4998
43       4503
55       4004
50       3942
44       3102
52       3064
60       2912
62       1485
63       1173
15       1143
Name: start_district_id, dtype: int64
49     139557
51     139421
48     132998
52     132301
105    131759
50     131642
108    119419
106    118

In [7]:
# Now try to generate the gap data. We're looking for the orders which were not answered by a driver.
# Hence we extract the orders with missing driver and count the number of them in each distrct and time slot.
# The number of the not answered orders should be the gap we're looking for

gaporders = orders[pd.isnull(orders['driver_id'])]
gap = pd.DataFrame({'gap' : gaporders.groupby( ['day', 'start_district_id', 'time_slot'] ).size()}).reset_index()
gap['key'] = gap.day.astype(str).str.cat(gap.start_district_id.astype(str), sep='_').str.cat(gap.time_slot.astype(str), sep='_')
gap["district_id"] = gap['start_district_id']

In [8]:
# Create a new DataFrame to save the useful information
time_slot = pd.DataFrame({'stat' : orders.groupby(["time_slot"]).size()})
szone = pd.DataFrame({'stat' : orders.groupby(["start_district_id"]).size()})
ezone = pd.DataFrame({'stat' : orders.groupby(["dest_district_id"]).size()})
day = pd.DataFrame({'stat' : orders.groupby(["day"]).size()})
print (len(time_slot),len(szone), len(ezone), len(day))
in_d = pd.DataFrame(columns=['day', 'start_district_id', 'time_slot',"gap"])
n = 1
for d in day.index.values:    
    date = d
    print('day: '+ str(d))
    for z in szone.index.values:
        start = z
        for t in time_slot.index.values:
            time = t
            in_d.loc[n]=[date, start, time, 0.0]
            n+=1

(144, 66, 66, 20)
day: 2
day: 3
day: 4
day: 5
day: 6
day: 7
day: 8
day: 9
day: 10
day: 11
day: 12
day: 13
day: 14
day: 15
day: 16
day: 17
day: 18
day: 19
day: 20
day: 21


In [13]:
in_d

,day,start_district_id,time_slot,gap
1,2,1,0,0
2,2,1,1,0
3,2,1,2,0
4,2,1,3,0
5,2,1,4,0
6,2,1,5,0
7,2,1,6,0
8,2,1,7,0
9,2,1,8,0
10,2,1,9,0


In [14]:
# Create a new column called key which combines day, start district id and time slot. 
in_d['key'] = in_d["day"]
in_d['key'] = in_d.day.astype(int).astype(str).str.cat(in_d.start_district_id.astype(str), sep='_').str.cat(in_d.time_slot.astype(int).astype(str), sep='_')
in_d["district_id"] = in_d["start_district_id"]

In [15]:
# Generate demand, supply and future supply (which depends on the destination of each order). Our naive model 
# will be based on these variables. Future supply will be an interesting attribute to dig in,

demand = pd.DataFrame({'demand' : orders.groupby( ['day', 'start_district_id', 'time_slot'] ).size()}).reset_index()
supply = pd.DataFrame({'demand' : orders.groupby( ['day', 'driver_id', 'time_slot'] ).size()}).reset_index()
fsupply = pd.DataFrame({'future_supply' : orders.groupby( ['day', 'dest_district_id', 'time_slot'] ).size()}).reset_index()
demand["district_id"] = demand['start_district_id']
fsupply["district_id"] = fsupply['dest_district_id']

In [16]:
# fill in the table with the information generated.

def join_value(left, right, key):
    left['key'] = left.day.astype(int).astype(str).str.cat(left.district_id.astype(int).astype(str), sep='_').str.cat(left.time_slot.astype(int).astype(str), sep='_')
    right['key'] = right.day.astype(int).astype(str).str.cat(right.district_id.astype(int).astype(str), sep='_').str.cat(right.time_slot.astype(int).astype(str), sep='_')
    left_a = left.set_index(key)
    right_a = right.set_index(key)

    res = left_a.loc[:, left_a.columns.union(right_a.columns)]
    res.update(right_a)
    return res

res = join_value(in_d, gap, "key")
res = join_value(res, demand, "key")
res = join_value(res, fsupply, "key")
res["time"] = res["time_slot"]/144*24
res.drop(["dest_district_id", "start_district_id"], axis=1,inplace=True) 
res["supply"] = res["demand"]-res["gap"]

print (res.sort_values(["gap"], ascending=False))


NameError: name 'np' is not defined

In [19]:
import numpy as np
temp_est = np.append(res["gap"][1:].as_matrix(),res["gap"][len(gap)-1])
res["gap_next"] = temp_est

In [53]:
res.loc()

,L1,L2,L3,L4,day,demand,district_id,fri,future_supply,gap,...,gap_slot,mon,sat,sun,supply,thu,time,time_slot,tue,wed
key,,,,,,,,,,,,,,,,,,,,,
2_1_0,NaN,NaN,NaN,NaN,2,88,1,False,62,3,...,0,False,True,False,85,False,0.000000,0,False,False
2_1_1,1309,244,93,76,2,97,1,False,52,7,...,1,False,True,False,90,False,0.166667,1,False,False
2_1_2,1274,196,59,46,2,75,1,False,53,5,...,1,False,True,False,70,False,0.333333,2,False,False
2_1_3,1197,213,67,45,2,60,1,False,50,2,...,0,False,True,False,58,False,0.500000,3,False,False
2_1_4,1236,189,36,51,2,72,1,False,52,7,...,1,False,True,False,65,False,0.666667,4,False,False
2_1_5,1197,184,35,47,2,86,1,False,54,18,...,3,False,True,False,68,False,0.833333,5,False,False
2_1_6,1269,179,32,39,2,59,1,False,32,5,...,1,False,True,False,54,False,1.000000,6,False,False
2_1_7,1217,140,61,37,2,44,1,False,23,2,...,0,False,True,False,42,False,1.166667,7,False,False
2_1_8,1189,127,46,50,2,42,1,False,33,1,...,0,False,True,False,41,False,1.333333,8,False,False


In [35]:
# Use path to data files.
traffic_path = "C:\\Users\\danny\\Di-Tech\\season_2\\training_data\\traffic_data\\" 
_, _, filenames = next(walk(traffic_path), (None, None, []))

mSeq = len(filenames)
frame = pd.DataFrame()
list_ = []
print filenames
# Uncomment these lines to import all the data 
for i in range(mSeq-1):
    # White Space as Delim, import all the data file.
    df = pd.read_csv(str(traffic_path) + filenames[i+1],  delim_whitespace = 1, dtype = str, header = None) 
    list_.append(df)

traffic = pd.concat(list_)

# Only import the data of the first day
# orders = pd.read_csv(str(order_path) + filenames[1], delim_whitespace = 1, dtype = str,  header = None) 
# Rename all the columns.
traffic.columns = ['district_hash', 'L1', 'L2', 
                  'L3', 'L4', 
                  'date',"time"]
# Convert the Date and Time column in orders from an object to a datetime value. 
# This makes it easier to work with as a date.
# Extract the year and month from from date_time, and assign them to their own columns.
traffic['datetime'] = pd.to_datetime(traffic['date'] + ' ' + traffic['time'])
traffic = traffic.drop('date', 1)
traffic = traffic.drop('time', 1)
traffic['day'] = traffic["datetime"].dt.day
# Every 10 minutes is considered as a time slot. It can be calculated by following fomula
traffic['time_slot'] = traffic["datetime"].dt.hour*6 + traffic["datetime"].dt.minute//10

"""Categorize the start and destination with its id in """

# Import the hash id into a dictionary
cluster_path = "season_2/training_data/cluster_map/"
cluster = {}
with open(cluster_path + "cluster_map") as f:
    for line in f:
       (key, val) = line.split()
       cluster[str(key)] = val
        
# Update the start and destination district ID
trafficL1 = traffic["L1"].str.split(":", expand=True)
traffic['district_id'] = traffic['district_hash'].map(cluster)
traffic["key"]=traffic.day.astype(str).str.cat(traffic.district_id.astype(str), sep='_').str.cat(traffic.time_slot.astype(str), sep='_')
trafficdf = traffic[["key","district_id", "day","time_slot"]]
trafficdf.loc[:,"L1"]=traffic["L1"].str.split(":", expand=True)[1]
trafficdf.loc[:,"L2"]=traffic["L2"].str.split(":", expand=True)[1]
trafficdf.loc[:,"L3"]=traffic["L3"].str.split(":", expand=True)[1]
trafficdf.loc[:,"L4"]=traffic["L4"].str.split(":", expand=True)[1]


['traffic_data_2016-01-01', 'traffic_data_2016-01-02', 'traffic_data_2016-01-03', 'traffic_data_2016-01-04', 'traffic_data_2016-01-05', 'traffic_data_2016-01-06', 'traffic_data_2016-01-07', 'traffic_data_2016-01-08', 'traffic_data_2016-01-09', 'traffic_data_2016-01-10', 'traffic_data_2016-01-11', 'traffic_data_2016-01-12', 'traffic_data_2016-01-13', 'traffic_data_2016-01-14', 'traffic_data_2016-01-15', 'traffic_data_2016-01-16', 'traffic_data_2016-01-17', 'traffic_data_2016-01-18', 'traffic_data_2016-01-19', 'traffic_data_2016-01-20', 'traffic_data_2016-01-21']


C:\Users\danny\Anaconda2\lib\site-packages\pandas\core\indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\danny\Anaconda2\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [36]:
trafficdf

,key,district_id,day,time_slot,L1,L2,L3,L4
0,2_66_141,66,2,141,225,44,2,5
1,2_66_55,66,2,55,321,65,7,16
2,2_64_131,64,2,131,519,69,21,7
3,2_64_38,64,2,38,237,20,11,8
4,2_64_5,64,2,5,480,40,13,8
5,2_64_83,64,2,83,478,105,34,7
6,2_2_104,2,2,104,596,196,27,23
7,2_2_19,2,2,19,495,25,11,15
8,2_2_56,2,2,56,667,108,23,15
9,2_15_24,15,2,24,10,0,0,0


In [43]:
res = join_value(res, trafficdf, "key")

C:\Users\danny\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [46]:
res.to_csv("train_traffic.csv")

In [45]:
res['fri'] = res['day']==1
res['sat'] = res['day']==2
res['sun'] = res['day']==3
res['mon'] = res['day']==4
res['tue'] = res['day']==5
res['wed'] = res['day']==6
res['thu'] = res['day']==7

res['gap_slot'] = res['gap']
res['gap_max'] = 100
res['gap_slot'] = res[['gap_slot', 'gap_max']].min(axis=1)
res['gap_slot'] = res['gap_slot']//5

In [47]:
input = pd.DataFrame(res[['day', 'district_id', 'gap', 'time_slot',"L1","L2","L3","L4"]])
output = pd.DataFrame(res[['day', 'time_slot', 'gap', 'gap_slot',"L1","L2","L3","L4"]])

input = input.drop(input[input.time_slot == 143].index)
output = output.drop(output[output.time_slot == 0].index)
time_slot_dummies = pd.get_dummies(input['time_slot'])
for i in range(142):
    input['time_slot_'+str(i)] = time_slot_dummies[float(i)]
district_dummies = pd.get_dummies(input['district_id'])
for i in range(65):
    input['district_id_'+str(i+1)] = district_dummies[str(i+1)]
input['fri'] = input['day']==1
input['sat'] = input['day']==2
input['sun'] = input['day']==3
input['mon'] = input['day']==4
input['tue'] = input['day']==5
input['wed'] = input['day']==6
input.drop('district_id', axis=1, inplace=True)
input.drop('time_slot', axis=1, inplace=True)

In [ ]:
input

In [ ]:
output

In [48]:
# Convert the data we need into np array for training network
import numpy as np
train_x = input.as_matrix().astype(float)
train_y = output['gap'].as_matrix().astype(float)
train_y_class = output['gap_slot'].as_matrix().astype(float)

In [49]:
float(input.iloc[[1]]['gap'])-float(input.iloc[[0]]['gap'])

4.0

### Export the np array and dataframe as txt for future convenience

In [50]:
print(train_x.shape, train_y.shape, train_y_class.shape)

((188760L, 219L), (188760L,), (188760L,))


In [51]:
np.savetxt('train_x.txt', train_x, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')
np.savetxt('train_y.txt', train_y, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')
np.savetxt('train_y_class.txt', train_y_class, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')

In [52]:
res.to_csv('res.txt', sep = ' ')
input.to_csv('input.txt', sep=' ')
output.to_csv('output.txt', sep=' ')

### Direct read the modified dataframe from csv

In [ ]:
res = pd.read_csv('res.txt',  delim_whitespace = 1,) 
input = pd.read_csv('input.txt',  delim_whitespace = 1,) 
output = pd.read_csv('output.txt',  delim_whitespace = 1,) 

In [ ]:
input.drop('key', axis=1, inplace=True)
output.drop('key', axis=1, inplace=True)